# CE-40959: Deep Learning
## HW2 - CIFAR-10 Classification (Pytorch)

(18 points)

### Deadline: 23 Esfand

#### Name:
#### Student No.:


Please review `Pytorch Tutorial` notebook (materials of the TA classes) before coming to this notebook and you can use `pytorch.org` to learn how to use PyTorch classes and commands.

In this part you have to implement MLP for Classification of CIFAR-10 dataset. 

PyTorch provides the elegantly designed modules and classes `torch.nn`, `torch.optim` , `Dataset` , and `DataLoader` to help you create and train neural networks. In this homework you use them for your implementations.

In [0]:
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
import torchvision
import numpy as 
# test for save

#### 3.1. Load Data:

Complete the followed cell for data loading. 
In this cell you have to normalize, split and shuffle data for learning.

In [0]:
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

trainloader = None
validationloader = None
testloader = None
##################################################################################
# TODO: Use 'torchvision.datasets.CIFAR-10' class for loading CIFAR-10 dataset.  #
# This dataset has 50000 data for training and 10000 data for test and every     #
# data has shape (3*32*32).                                                      #
# Also Use 'torchvision.transforms.Compose' for common image transformations     #
# such as normalization and use 'torch.utils.data.DataLoader' class that it      #
# represents a Python iterable over a dataset and divides data to Batches.       #
# Then Split data into 3 part: Train, Validation and Test. Finally,              #
# save iterable data in 'trainloader', 'validationloader', 'testloader'.         #
##################################################################################

batch_size_train = 64
batch_size_test = 64

valid_size = 0.2

transform = transforms.Compose([
                                transforms.ToTensor(), 
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                ])

train_validatation_set = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform
    )

num_train = len(train_validatation_set)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))

trainloader = torch.utils.data.DataLoader(
    train_validatation_set, batch_size=batch_size_train, 
    sampler=SubsetRandomSampler(indices[split:])
    )

validationloader = torch.utils.data.DataLoader(
    train_validatation_set, batch_size=batch_size_train, 
    sampler=SubsetRandomSampler(indices[:split])
    )


test_set = torchvision.datasets.CIFAR10(
    root='./data', train=False,download=True, transform=transform
    )

testloader = torch.utils.data.DataLoader(
    test_set, batch_size=batch_size_test , shuffle=False, num_workers=2
    )

##################################################################################
#                               End of your code                                 #
##################################################################################

classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

#### 3.2. Load Data Test:

In [0]:
############################################################
# Run the following code an check the size of each batch   #
############################################################
examples = enumerate(trainloader)
batch_idx, (example_data, example_targets) = next(examples)
print('The size and type of each batch in ''trainloader'' is:')
print(example_data.size())
print(type(example_data))
examples = enumerate(testloader)
batch_idx, (example_data, example_targets) = next(examples)
print('\nThe size and type of each batch in ''testloader'' is:')
print(example_data.size())
print(type(example_data))

In [0]:
#####################################################################
# Run the following code and see some of the samples in the dataset #
#####################################################################

dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images:
for i in range(4):
    img=torchvision.utils.make_grid(images[i])
    ###########################################################
    #  If you normalize data , here unnormalize them to see   # 
    #  clear them.                                            #
    ###########################################################
    m = 0.5
    s = 0.5
    img = img * s + m    # unnormalize
    ###########################################################
    #                   End of your code                      #
    ###########################################################
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1,2, 0)))
    plt.title("Target Labels: {}".format(classes[labels[i]]))
    plt.axis('off')
    plt.show()
    

#### 3.3. Network Design:
Design the layer of your network and select proper hyperparameter. 



In [0]:
import torch.nn as nn

######################################################################
# TODO: Use 'torch.nn' module to design your network for CIFAR-10    #
# classification. You have to implement the structure of MLP for it. #
# In your design you don't have any limitation and you can use       #
# Batch-norm layers, Drop-out layers and etc for generalization      #
# improvement (if needed). Use classes and modules from 'torch.nn'.  #
# In the following code, the 'MLP' class is your MLP network and     #
# this class is inherited from nn.Module, so you can benefit         #
# properties of the 'nn.Module'.You may complete '__init__()'        #
# constructor by some classes like 'nn.ReLU()' or 'nn.Linear()'      #
# to use them in the forward pass of your network.                   #
######################################################################
  
class MLP(nn.Module):
    def __init__(self, D_in, H1, H2, D_out):
        super(MLP, self).__init__()
        
        self.linear1 = torch.nn.Linear(D_in, H1)
        self.batch1 = torch.nn.BatchNorm1d(H1)
        self.relu1 = torch.nn.ReLU()

        self.linear2 = torch.nn.Linear(H1, H2)
        self.batch2 = torch.nn.BatchNorm1d(H2)
        self.relu2 = torch.nn.ReLU()
        
        self.linear3 = torch.nn.Linear(H2, D_out)

    def forward(self, x):
        out = self.linear1(x)
        out = self.batch1(out)
        out = self.relu1(out)

        out = self.linear2(out)
        out = self.batch2(out)
        out = self.relu2(out)
        
        out = self.linear3(out)
        return out

######################################################################
#                          End of your code                          #
######################################################################

#### 3.4. Optimization Algorithm:

In [0]:
import torch.optim as optim

#############################################################################
# TODO: Use a Classification Cross-Entropy loss.Then, use 'torch.optim'     #
# module to optimize Cross-Entropy loss. You should select a optimization   #
# algorithm and its hyperparameters like learning rate.                     #
#############################################################################
net = MLP(3072, 100, 50, len(classes))
learning_rate = 1e-3
criterion = torch.nn.functional.cross_entropy
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

#############################################################################
#                             End of your code                              #
#############################################################################

#### 3.5. Training:
You have to tweak `hidden_dim`, `leanirng_rate`, `weight_scale`, `num_epochs` and `reg` and etc to get a validation accuracy above 50%.

In [0]:
#######################################################
# TODO: Feed the inputs data to the MLP network and   #
# optimize Cross-Entropy loss by using target labels. #
# Then update weights and biases.                     #
#######################################################

num_epochs=10
num_batchs = len(trainloader)
for epoch in range(num_epochs):
    total_train=0
    correct_train=0
    running_loss = 0.0
    for batch, data in enumerate(trainloader, 0):
        inputs, labels = data
        
        (b_size, n_channel, w, h) = inputs.shape
        inputs = inputs.reshape(b_size, n_channel * w * h)

        # zero the parameter gradients:
        optimizer.zero_grad()
        pass

        # forward pass:
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        pass

        # backward pass:
        loss.backward()
        pass

        # optimization:
        optimizer.step()

        pass
        #############################################
        #           End of your code                #
        #############################################
        

        # Results: 
        running_loss += loss.item()

        total_train += labels.size(0)
        _, predicted_train = torch.max(outputs.data, 1)
        correct_train += (predicted_train == labels).sum().item()

        if batch % (num_batchs/10) == ((num_batchs/10) -1):
            print('[Batch %d / %d] loss: %.3f' %
                  (batch + 1, num_batchs, running_loss / (num_batchs/10)))
            running_loss = 0.0
            torch.save(net.state_dict(), './model.pth')
            torch.save(optimizer.state_dict(), './optimizer.pth')
    correct = 0
    total = 0
    with torch.no_grad():
        for data in validationloader:
            images, labels = data

            (b_size, n_channel, w, h) = images.shape
            images = images.reshape(b_size, n_channel * w * h)
            
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_acc = correct / total
    train_acc = correct_train / total_train
    print('(Epoch %d / %d) train acc: %.2f%%; val_acc: %.2f%%' % (
          epoch+1, num_epochs, 100*train_acc, 100*val_acc))

#### 3.6. Test: 
Run the following cell and test your network.

In [0]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data

        (b_size, n_channel, w, h) = images.shape
        images = images.reshape(b_size, n_channel * w * h)

        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
test_acc = correct / total
print('Accuracy of the network on the test images: %2f %%' % (100 * test_acc ))

In [0]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data

        (b_size, n_channel, w, h) = images.shape
        images = images.reshape(b_size, n_channel * w * h)

        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))